In [1]:
import config
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import cv2

In [2]:
imagem = cv2.imread("imagen.png")
#cv2.imshow("Original", imagem)

In [11]:
imagem.shape

(906, 958, 3)

In [35]:
fatia = imagem[430:480, 650:770 ]
cv2.imwrite("img.png", fatia)


True

In [27]:
fatia

array([[[230, 226, 223],
        [229, 225, 222],
        [231, 227, 223],
        ...,
        [221, 216, 212],
        [224, 220, 215],
        [224, 219, 216]],

       [[229, 225, 222],
        [228, 223, 221],
        [227, 222, 219],
        ...,
        [218, 213, 209],
        [219, 214, 210],
        [222, 218, 213]],

       [[228, 223, 221],
        [226, 222, 219],
        [225, 220, 217],
        ...,
        [216, 210, 206],
        [217, 211, 207],
        [218, 212, 208]],

       ...,

       [[223, 218, 215],
        [221, 216, 212],
        [219, 214, 210],
        ...,
        [252, 252, 252],
        [249, 249, 248],
        [241, 240, 238]],

       [[224, 220, 217],
        [223, 218, 214],
        [221, 216, 212],
        ...,
        [209, 202, 197],
        [210, 203, 199],
        [211, 205, 200]],

       [[226, 221, 218],
        [224, 220, 217],
        [223, 218, 215],
        ...,
        [212, 205, 201],
        [213, 207, 202],
        [214, 208, 204]]

In [19]:
options = webdriver.ChromeOptions()
#options.add_argument("--headless")
driver = webdriver.Chrome(
        config.CHROMEDRIVER,
        options=options
        )


C:\Users\Usuário\AppData\Local\Temp\ipykernel_15956\1766906669.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [20]:
url = 'http://pergamum.museu-goeldi.br:8080/pergamumweb/home_geral/login.jsp'
driver.get(url)

In [32]:
img = driver.find_element(By.ID, 'imgCaptcha')
img.location

{'x': 606, 'y': 428}

In [33]:
img.get_attribute('src')

'http://pergamum.museu-goeldi.br:8080/pergamumweb/kaptcha.jpg'

In [28]:
driver.save_screenshot('imagen.png')

True

In [10]:
data = 'rs=ajax_dados_acervo&rst=&rsrnd=1664285637996&rsargs[]=30850&rsargs[]='

In [11]:
response = httpx.post(
    'http://pergamum.museu-goeldi.br/pergamum/biblioteca/index.php', 
    cookies=cookies, 
    headers=headers, 
    data=data, 
    verify=False)
response

<Response [200 OK]>

In [13]:
response = httpx.get('http://pergamum.museu-goeldi.br/pergamum/biblioteca/index.php?codAcervo=35191')
response

<Response [200 OK]>